In [1]:
import numpy as np
import pandas as pd
import math

# Risk free spot rate (continuously compounded) to discounted factor

In [125]:
def discount(spot_rate, spot=True, t=1):
    """
    or discount factor: spot_rate=False for reverse
    """
    if spot==True:
        return math.exp(-spot_rate*t)
    else: return -np.log(spot_rate)/t

# Bonds

### ZCB using credit spread

In [3]:
import numpy as np

def cs(PD, Rec, rf, T, N=100):
    return -(1/T)*np.log((1-PD)*(1-Rec)+Rec)
cs(0.045, 0.4, 0.018, 3)

In [15]:
def price_ZCB_creditspread(PD, Rec, rf, T):
    def cs(PD, Rec, rf, T, N=100):
        return -(1/T)*np.log((1-PD)*(1-Rec)+Rec)
    return np.exp((rf+cs(PD, Rec, rf, T))*-1*T)
#V_D = np.exp((rf+cs(PD, R, rf, T))*-1*3)
#V_D

In [17]:
price_ZCB_creditspread(PD=0.045, Rec=0.4, rf=0.018, T=3)

0.9218514396262497

### Pricing risk coupon bonds - from survival probilities

In [2]:
import math

In [8]:
def price_risky_coupon_bond(term_structure, c, m, Rec, N=100):
    """
    term stucture: nest dict of {dict: terms, keys: spot_rate, survival_prob}
    """
    def discount(spot_rate, spot=True, t=1):
        """
        or discount factor: spot_rate=False for reverse
        """
        if spot==True:
            return math.exp(-spot_rate*t)
        else: return -np.log(spot_rate)/t
    def sum1(term_structure):
        sum = 0
        for x in term_structure:
            sum += discount(term_structure[x]['spot_rate'], t=x)*term_structure[x]['survival_prob']
        return sum

    def sum2(term_structure):
        sum = 0
        for i,x in enumerate(term_structure):
            if i == 0:
                sum += discount(term_structure[x]['spot_rate'], t=x)*\
                (1-term_structure[x]['survival_prob'])
            else:
                sum += discount(term_structure[x]['spot_rate'], t=x)*\
                (term_structure[list(term_structure)[i-1]]['survival_prob']-\
                 term_structure[x]['survival_prob'])
        return sum
    return (c/m)*N*sum1(term_structure) \
    + term_structure[list(term_structure)[-1]]['survival_prob']\
    * discount(term_structure[list(term_structure)[-1]]['spot_rate'],t=list(term_structure)[-1])*N\
    + N*Rec*sum2(term_structure)

In [9]:
#eg 1
term_structure = {
    0.5: {'spot_rate': 0.025, 'survival_prob': 0.93},
    1: {'spot_rate': 0.03, 'survival_prob': 0.91}
}
price_risky_coupon_bond(term_structure, c=0.08, m=2, Rec=0.45)

99.50102576461744

In [10]:
#eg 2
term_structure = {
    0.5: {'spot_rate': 0.03, 'survival_prob': 0.98},
    1: {'spot_rate': 0.035, 'survival_prob': 0.94}
}
price_risky_coupon_bond(term_structure, c=0.06, m=2, Rec=0.4, N=1)

0.9871920372218514

### Bootstapping implied survival probability - from bond prices

In [119]:
#eg1
bonds = {
    1: {'c':0.04, 'p':0.97, 'd':0.99},
    2: {'c':0.05, 'p':0.96, 'd':0.98}
}
N = 1
Rec = 0.4
m =1

Q = ((bonds[1]['p']/N)-Rec*bonds[1]['d'])/(bonds[1]['d']*(1+bonds[1]['c']/m-Rec))
Q

0.9059343434343433

In [57]:

    
bonds = {
    0.5: {'c':0.04, 'p':0.95, 'd':discount(0.02, t=0.5)},
    1: {'c':0.05, 'p':0.96, 'd':discount(0.03, t=1)}
}
N = 1
Rec = 0.4
m = 2

In [67]:
bonds[0.5]['Q'] = ((bonds[0.5]['p']/bonds[0.5]['d'])-Rec)/(1+bonds[0.5]['c']/m-Rec)
bonds

{0.5: {'c': 0.04, 'p': 0.95, 'd': 0.9900498337491681, 'Q': 0.9024962237579992},
 1: {'c': 0.05, 'p': 0.96, 'd': 0.9704455335485082}}

In [162]:
def implied_survival_probs(bonds, N=100, Rec=0.4, m=2, T=list(bonds)[-1]):
    for i,x in enumerate(bonds):
        if i == 0:
            bonds[x]['Q'] = ((bonds[x]['p']/bonds[x]['d'])-Rec)/(1+bonds[x]['c']/m-Rec)
        else:
            sums = np.zeros(2)
            sums[0] += bonds[list(bonds)[i-1]]['d']*\
            (Rec*(1-bonds[list(bonds)[i-1]]['Q'])+\
             (bonds[x]['c']/m)*bonds[list(bonds)[i-1]]['Q'])
            sums[1] += Rec*bonds[x]['d']*bonds[list(bonds)[i-1]]['Q']
            bonds[x]['Q'] = (bonds[list(bonds)[-1]]['p']-sum(sums))/(bonds[list(bonds)[-1]]['d']*((1+((bonds[list(bonds)[-1]]['c']/m))-Rec)))
    return bonds

implied_survival_probs(bonds) 

{1: {'c': 0.04, 'p': 0.97, 'd': 0.99, 'Q': 0.9351580319322255},
 2: {'c': 0.05, 'p': 0.96, 'd': 0.98, 'Q': 0.8891354625916837}}

In [163]:
#CW1
bonds = {
    0.5: {'c':0.04, 'p':0.95, 'd':discount(0.02, t=0.5)},
    1: {'c':0.05, 'p':0.96, 'd':discount(0.03, t=1)}
}
implied_survival_probs(bonds)

{0.5: {'c': 0.04, 'p': 0.95, 'd': 0.9900498337491681, 'Q': 0.9024962237579992},
 1: {'c': 0.05, 'p': 0.96, 'd': 0.9704455335485082, 'Q': 0.9046884374385575}}

In [170]:
# Ballotta et al pg.17
bonds = {
    1: {'c':0.04, 'p':0.97, 'd':0.99},
    2: {'c':0.05, 'p':0.96, 'd':0.98},
}

implied_survival_probs(bonds, m=1)

{1: {'c': 0.04, 'p': 0.97, 'd': 0.99, 'Q': 0.9059343434343433},
 2: {'c': 0.05, 'p': 0.96, 'd': 0.98, 'Q': 0.8206907180121464}}

# CDS - credit default swap

### Spread from survival probability

In [18]:
def CDS_spread_computation(term_structure, Rec=0.4):
    for i,x in enumerate(term_structure):
        if i == 0:
            term_structure[x]['alpha'] = 0
        else:
            term_structure[x]['alpha'] = x-list(term_structure)[i-1]
            term_structure[x]['A'] = term_structure[x]['d']*term_structure[x]['Q']*term_structure[x]['alpha']
            term_structure[x]['Ahat'] = term_structure[x]['d']*term_structure[list(term_structure)[i-1]]['Q']*term_structure[x]['alpha']
            term_structure[x]['B'] = term_structure[x]['d']*term_structure[x]['Q']
            term_structure[x]['Bhat'] = term_structure[x]['d']*term_structure[list(term_structure)[i-1]]['Q']
    df = pd.DataFrame(term_structure).T
    df.index.name = 'T'
    return df
def CDS_spread(term_structure, Rec=0.4):
    df = CDS_spread_comp(term_structure, Rec)
    R = ((1-Rec)*(df.sum()['Bhat']-df.sum()['B']))/(0.5*(df.sum()['Ahat']+df.sum()['A']))
    return 'CDS spread (R) = {}'.format(round(R,5))

In [124]:
# Lecture 3: S32 example
term_structure ={
    0: {'d':1, 'Q': 1},
    0.25: {'d':0.99, 'Q':0.999},
    0.5: {'d':0.98, 'Q':0.994},
    0.75: {'d':0.97, 'Q':0.987},
    1: {'d':0.96, 'Q':0.977},
}
CDS_spread_comp(term_structure), CDS_spread(term_structure)

(         d      Q  alpha         A      Ahat        B     Bhat
 T                                                             
 0.00  1.00  1.000   0.00       NaN       NaN      NaN      NaN
 0.25  0.99  0.999   0.25  0.247252  0.247500  0.98901  0.99000
 0.50  0.98  0.994   0.25  0.243530  0.244755  0.97412  0.97902
 0.75  0.97  0.987   0.25  0.239347  0.241045  0.95739  0.96418
 1.00  0.96  0.977   0.25  0.234480  0.236880  0.93792  0.94752,
 'CDS spread (R) = 0.01382')

### bootstrapping survival probabilities from CDS spread

In [121]:
def bootstrap_survival_CDS(spreads, Rec=0.4):
    for i,x in enumerate(spreads):
        info = spreads
        if i == 0:
            info[x]['alpha'] = 0
            info[x]['Q'] = 1
        else:
            info[x]['alpha']= x-list(info)[i-1]
            if i == 1:
                info[x]['Q']= ((1-Rec)-info[x]['cds']*info[x]['alpha']/2)/\
                                        ((info[x]['cds']*info[x]['alpha']/2)+(1-Rec))
                info[x]['A']= info[x]['alpha']*info[x]['d']*info[x]['Q']
                info[x]['Ahat']= info[x]['alpha']*info[x]['d']
                info[x]['B']= info[x]['d']*info[x]['Q']
                info[x]['Bhat']= info[x]['d']
            else:
                info[x]['Ahat']= info[x]['alpha']*info[x]['d']*info[list(info)[i-1]]['Q']+quotes[list(info)[i-1]]['Ahat']
                info[x]['Bhat']= info[x]['d']*info[list(info)[i-1]]['Q']+info[list(info)[i-1]]['Bhat']
                info[x]['Q']= ((1-Rec)*(info[x]['Bhat']-info[list(info)[i-1]]['B'])\
                    - 0.5*info[x]['cds']*(info[x]['Ahat']+info[list(info)[i-1]]['A']))\
                    /(info[x]['d']*(0.5*info[x]['cds']*info[x]['alpha']+(1-Rec)))
                info[x]['A']= info[x]['alpha']*info[x]['d']*info[x]['Q']+info[list(info)[i-1]]['A']
                info[x]['B']= info[x]['d']*info[x]['Q']+info[list(info)[i-1]]['B']
    df = pd.DataFrame(info).T
    df.index.name='T'
    df = df[[col for col in df if col != 'Q']+['Q']]
    return df


In [123]:
# Lecture 3: S35 example
CDS_spreads = {
    0: {'d':1, 'cds':0},
    1: {'d':0.987, 'cds': 0.020},
    2: {'d':0.98, 'cds': 0.025},
    3: {'d':0.975, 'cds': 0.031},
    4: {'d':0.97, 'cds': 0.037},
    5: {'d':0.963, 'cds': 0.045}
}
bootstrap_survival_CDS(CDS_spreads)

,d,cds,alpha,A,Ahat,B,Bhat,Q
T,,,,,,,,
0,1.000,0.000,0.0,NaN,NaN,NaN,NaN,1.000000
1,0.987,0.020,1.0,0.954639,0.987000,0.954639,0.987000,0.967213
2,0.980,0.025,1.0,1.855895,1.934869,1.855895,1.934869,0.919648
3,0.975,0.031,1.0,2.688915,2.831526,2.688915,2.831526,0.854379
4,0.970,0.037,1.0,3.441308,3.660274,3.441308,3.660274,0.775664
5,0.963,0.045,1.0,4.088643,4.407238,4.088643,4.407238,0.672206


In [126]:
# CW1
CDS_spreads = {
    0: {'d':1, 'cds':0},
    1: {'d':discount(0.04,t=1), 'cds': 0.02},
    2: {'d':discount(0.05,t=2), 'cds': 0.03},
    3: {'d':discount(0.053, t=3), 'cds': 0.032},
    4: {'d':discount(0.062, t=4), 'cds': 0.045},
}
bootstrap_survival_CDS(CDS_spreads)

,d,cds,alpha,A,Ahat,B,Bhat,Q
T,,,,,,,,
0,1.000000,0.000,0.0,NaN,NaN,NaN,NaN,1.000000
1,0.960789,0.020,1.0,0.929288,0.960789,0.929288,0.960789,0.967213
2,0.904837,0.030,1.0,1.745762,1.862171,1.745762,1.835960,0.902343
3,0.852996,0.032,1.0,2.467727,2.704564,2.467727,2.605655,0.846388
4,0.780360,0.045,1.0,3.021872,3.492013,3.021872,3.266142,0.710114


# CVA

Let us consider a 1 year forward contract on a commodity, which does not pay
implied dividends (ie the convenience yield is zero). Let us set the risk-free rate at
3%. The forward price is determined according to the usual no-arbitrage
cash-and-carry formula

In [171]:
S_t = 100
rf = 0.03
T = 1

In [173]:
F_tT = S_t*math.exp(rf*T)

In [174]:
F_tT

103.0454533953517